In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
%cd /content/drive/My Drive

/content/drive/My Drive


In [0]:
# Importing Libraries

In [0]:
import pandas as pd
import numpy as np

In [0]:
# Activities are the class labels
# It is a 6 class classification
ACTIVITIES = {
    0: 'WALKING',
    1: 'WALKING_UPSTAIRS',
    2: 'WALKING_DOWNSTAIRS',
    3: 'SITTING',
    4: 'STANDING',
    5: 'LAYING',
}

# Utility function to print the confusion matrix
def confusion_matrix(Y_true, Y_pred):
    Y_true = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_true, axis=1)])
    Y_pred = pd.Series([ACTIVITIES[y] for y in np.argmax(Y_pred, axis=1)])

    return pd.crosstab(Y_true, Y_pred, rownames=['True'], colnames=['Pred'])

### Data

In [0]:
# Data directory
DATADIR = 'UCI_HAR_Dataset'

In [0]:
# Raw data signals
# Signals are from Accelerometer and Gyroscope
# The signals are in x,y,z directions
# Sensor signals are filtered to have only body acceleration
# excluding the acceleration due to gravity
# Triaxial acceleration from the accelerometer is total acceleration
SIGNALS = [
    "body_acc_x",
    "body_acc_y",
    "body_acc_z",
    "body_gyro_x",
    "body_gyro_y",
    "body_gyro_z",
    "total_acc_x",
    "total_acc_y",
    "total_acc_z"
]

In [0]:
# Utility function to read the data from csv file
def _read_csv(filename):
    return pd.read_csv(filename, delim_whitespace=True, header=None)

# Utility function to load the load
def load_signals(subset):
    signals_data = []

    for signal in SIGNALS:
        filename = f'UCI_HAR_Dataset/{subset}/Inertial Signals/{signal}_{subset}.txt'
        signals_data.append(
            _read_csv(filename).as_matrix()
        ) 

    # Transpose is used to change the dimensionality of the output,
    # aggregating the signals by combination of sample/timestep.
    # Resultant shape is (7352 train/2947 test samples, 128 timesteps, 9 signals)
    return np.transpose(signals_data, (1, 2, 0))

In [0]:

def load_y(subset):
    """
    The objective that we are trying to predict is a integer, from 1 to 6,
    that represents a human activity. We return a binary representation of 
    every sample objective as a 6 bits vector using One Hot Encoding
    (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html)
    """
    filename = f'UCI_HAR_Dataset/{subset}/y_{subset}.txt'
    y = _read_csv(filename)[0]

    return pd.get_dummies(y).as_matrix()

In [0]:
def load_data():
    """
    Obtain the dataset from multiple files.
    Returns: X_train, X_test, y_train, y_test
    """
    X_train, X_test = load_signals('train'), load_signals('test')
    y_train, y_test = load_y('train'), load_y('test')

    return X_train, X_test, y_train, y_test

In [0]:
# Importing tensorflow
np.random.seed(42)
import tensorflow as tf
tf.set_random_seed(42)

In [0]:
# Configuring a session
session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1
)

In [0]:
# Import Keras
from keras import backend as K
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

Using TensorFlow backend.


In [0]:
# Importing libraries
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers.core import Dense, Dropout

In [0]:
# Initializing parameters
epochs = 30
batch_size = 16
n_hidden = 32

In [0]:
# Utility function to count the number of classes
def _count_classes(y):
    return len(set([tuple(category) for category in y]))

In [0]:
# Loading the train and test data
X_train, X_test, Y_train, Y_test = load_data()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


In [0]:
timesteps = len(X_train[0])
input_dim = len(X_train[0][0])
n_classes = _count_classes(Y_train)

print(timesteps)
print(input_dim)
print(len(X_train))

128
9
7352


- Defining the Architecture of LSTM

## 1- Layer LSTM

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(n_hidden, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
# Compiling the model
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/30




7352/7352 [==============================] - 29s 4ms/step - loss: 1.3389 - acc: 0.4314 - val_loss: 1.1651 - val_acc: 0.4747
Epoch 2/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.9892 - acc: 0.5755 - val_loss: 0.9336 - val_acc: 0.6108
Epoch 3/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.8011 - acc: 0.6517 - val_loss: 0.7875 - val_acc: 0.6121
Epoch 4/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.6928 - acc: 0.6710 - val_loss: 0.7502 - val_acc: 0.5948
Epoch 5/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.6376 - acc: 0.6949 - val_loss: 0.7110 - val_acc: 0.6668
Epoch 6/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.6068 - acc: 0.7119 - val_loss: 0.6957 - val_acc: 0.7139
Epoch 7/30
7352/7352 [====================

In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 510        0  ...                   0                27
SITTING                  1      374  ...                   1                17
STANDING                 0       73  ...                   0                17
WALKING                  0        0  ...                  27                41
WALKING_DOWNSTAIRS       0        0  ...                 414                 6
WALKING_UPSTAIRS         0        0  ...                  12               458

[6 rows x 6 columns]


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 302us/step


In [0]:
score

[0.5549750696317388, 0.8907363420427553]

- With a simple 2 layer architecture we got 89.07% accuracy and a loss of 0.55
- We can further imporve the performace with Hyperparameter tuning

## Model- 1(LSTM Units=32,dropout=0.5)

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(32, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 32)                5376      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 198       
Total params: 5,574
Trainable params: 5,574
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 28s 4ms/step - loss: 1.3445 - acc: 0.4415 - val_loss: 1.2233 - val_acc: 0.4581
Epoch 2/30
7352/7352 [==============================] - 28s 4ms/step - loss: 1.1281 - acc: 0.5061 - val_loss: 1.1085 - val_acc: 0.5633
Epoch 3/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.8849 - acc: 0.6329 - val_loss: 0.9060 - val_acc: 0.6427
Epoch 4/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.7139 - acc: 0.7004 - val_loss: 0.7249 - val_acc: 0.7055
Epoch 5/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.7125 - acc: 0.7100 - val_loss: 0.6530 - val_acc: 0.7214
Epoch 6/30
7352/7352 [==============================] - 28s 4ms/step - loss: 0.5608 - acc: 0.7801 - val_loss: 0.5915 - val_acc: 0.7961
Epoch 7/30
7352/7352 [==============================] - 27s 4ms/step - loss: 0.4779 - acc: 0.8319 - val_loss: 0.5202 - val_acc: 0.8178
Epoch 8

In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 510        0  ...                   0                27
SITTING                  0      417  ...                   0                 1
STANDING                 0      108  ...                   0                 0
WALKING                  0        1  ...                  21                 5
WALKING_DOWNSTAIRS       0        0  ...                 408                 9
WALKING_UPSTAIRS         0        0  ...                   2               450

[6 rows x 6 columns]


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 320us/step


In [0]:
score

[0.32979796789985005, 0.9087207329487614]

##Model-2:LSTM units 48,dropout=0.5

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(48, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 48)                11136     
_________________________________________________________________
dropout_1 (Dropout)          (None, 48)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 294       
Total params: 11,430
Trainable params: 11,430
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Train on 7352 samples, validate on 2947 samples
Epoch 1/30




7352/7352 [==============================] - 86s 12ms/step - loss: 1.2732 - acc: 0.4558 - val_loss: 1.1303 - val_acc: 0.4611
Epoch 2/30
7352/7352 [==============================] - 82s 11ms/step - loss: 1.0337 - acc: 0.5359 - val_loss: 0.8911 - val_acc: 0.6437
Epoch 3/30
7352/7352 [==============================] - 83s 11ms/step - loss: 0.8027 - acc: 0.6431 - val_loss: 0.8240 - val_acc: 0.6617
Epoch 4/30
7352/7352 [==============================] - 83s 11ms/step - loss: 0.6917 - acc: 0.7274 - val_loss: 0.5926 - val_acc: 0.7679
Epoch 5/30
7352/7352 [==============================] - 83s 11ms/step - loss: 0.5369 - acc: 0.8041 - val_loss: 0.5703 - val_acc: 0.7913
Epoch 6/30
7352/7352 [==============================] - 83s 11ms/step - loss: 0.4457 - acc: 0.8521 - val_loss: 0.5444 - val_acc: 0.8005
Epoch 7/30
7352/7352 [==============

In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 510        0  ...                   0                27
SITTING                  1      400  ...                   0                 0
STANDING                 0       88  ...                   0                 0
WALKING                  0        0  ...                  10                10
WALKING_DOWNSTAIRS       0        0  ...                 410                 9
WALKING_UPSTAIRS         0        0  ...                   4               465

[6 rows x 6 columns]


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 6s 2ms/step


In [0]:
score

[0.2968081940932087, 0.9175432643366135]

## LSTM units 64,dropout=0.25

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.25))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 64)                18944     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 85s 12ms/step - loss: 1.2476 - acc: 0.4400 - val_loss: 1.1749 - val_acc: 0.4588
Epoch 2/30
7352/7352 [==============================] - 85s 12ms/step - loss: 0.9634 - acc: 0.5739 - val_loss: 0.8059 - val_acc: 0.6912
Epoch 3/30
7352/7352 [==============================] - 84s 11ms/step - loss: 0.6816 - acc: 0.7278 - val_loss: 0.6412 - val_acc: 0.8001
Epoch 4/30
7352/7352 [==============================] - 85s 12ms/step - loss: 0.4905 - acc: 0.8364 - val_loss: 0.5425 - val_acc: 0.8290
Epoch 5/30
7352/7352 [==============================] - 84s 11ms/step - loss: 0.4151 - acc: 0.8562 - val_loss: 0.4200 - val_acc: 0.8497
Epoch 6/30
7352/7352 [==============================] - 84s 11ms/step - loss: 0.2822 - acc: 0.9053 - val_loss: 0.3974 - val_acc: 0.8724
Epoch 7/30
7352/7352 [==============================] - 83s 11ms/step - loss: 0.2325 - acc: 0.9208 - val_loss: 0.3152 - val_acc: 0.8843


In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 511        0  ...                   0                26
SITTING                  2      377  ...                   1                 1
STANDING                 0       83  ...                   0                 0
WALKING                  0        0  ...                   2                19
WALKING_DOWNSTAIRS       0        0  ...                 391                 5
WALKING_UPSTAIRS         0        0  ...                   0               441

[6 rows x 6 columns]


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 6s 2ms/step


In [0]:
score

[0.4332898249001449, 0.8971835765184933]

##(64 layers,dropout=0.5)

In [0]:
# Initiliazing the sequential model
model = Sequential()
# Configuring the parameters
model.add(LSTM(64, input_shape=(timesteps, input_dim)))
# Adding a dropout layer
model.add(Dropout(0.5))
# Adding a dense output layer with sigmoid activation
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 64)                18944     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 390       
Total params: 19,334
Trainable params: 19,334
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 34s 5ms/step - loss: 1.2662 - acc: 0.4358 - val_loss: 1.1064 - val_acc: 0.4829
Epoch 2/30
7352/7352 [==============================] - 33s 5ms/step - loss: 0.9313 - acc: 0.5763 - val_loss: 0.8966 - val_acc: 0.5691
Epoch 3/30
7352/7352 [==============================] - 33s 5ms/step - loss: 0.8923 - acc: 0.5998 - val_loss: 0.8434 - val_acc: 0.5606
Epoch 4/30
7352/7352 [==============================] - 33s 4ms/step - loss: 0.7401 - acc: 0.6597 - val_loss: 0.9543 - val_acc: 0.5874
Epoch 5/30
7352/7352 [==============================] - 33s 5ms/step - loss: 0.6950 - acc: 0.7016 - val_loss: 0.7862 - val_acc: 0.6620
Epoch 6/30
7352/7352 [==============================] - 33s 5ms/step - loss: 0.6325 - acc: 0.7520 - val_loss: 0.5506 - val_acc: 0.7957
Epoch 7/30
7352/7352 [==============================] - 33s 4ms/step - loss: 0.4968 - acc: 0.8251 - val_loss: 0.4399 - val_acc: 0.8402
Epoch 8

In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 533        0  ...                   0                 4
SITTING                  0      430  ...                   0                 1
STANDING                 0      120  ...                   0                 2
WALKING                  0        0  ...                  27                17
WALKING_DOWNSTAIRS       0        0  ...                 416                 1
WALKING_UPSTAIRS         0        0  ...                   6               457

[6 rows x 6 columns]


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 1s 422us/step


In [0]:
score

[0.3611737099244576, 0.9155072955548015]

## Model-3: 2 LSTM Layers + larger dropout(0.7)

In [0]:
# code from https://keras.io/regularizers/
from keras.regularizers import L1L2
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
from keras.layers import LSTM , BatchNormalization
reg = L1L2(0.01, 0.01)

In [0]:
model = Sequential()
model.add(LSTM(132, input_shape=(timesteps, input_dim), kernel_initializer='glorot_normal' , return_sequences=True, bias_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(0.70))
model.add(LSTM(68))
model.add(Dropout(0.70))
model.add(Dense(n_classes, activation='sigmoid'))
model.summary()




Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128, 132)          74976     
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 132)          528       
_________________________________________________________________
dropout_1 (Dropout)          (None, 128, 132)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 68)                54672     
_________________________________________________________________
dropout_2 (Dropout)          (None, 68)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 414       
T

In [0]:
# Compiling the model
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
# Training the model
model.fit(X_train,
          Y_train,
          batch_size=batch_size,
          validation_data=(X_test, Y_test),
          epochs=epochs)

Train on 7352 samples, validate on 2947 samples
Epoch 1/30
7352/7352 [==============================] - 242s 33ms/step - loss: 0.0849 - acc: 0.9711 - val_loss: 0.1123 - val_acc: 0.9614
Epoch 2/30
7352/7352 [==============================] - 242s 33ms/step - loss: 0.0774 - acc: 0.9745 - val_loss: 0.1079 - val_acc: 0.9701
Epoch 3/30
7352/7352 [==============================] - 242s 33ms/step - loss: 0.0744 - acc: 0.9748 - val_loss: 0.1355 - val_acc: 0.9633
Epoch 4/30
7352/7352 [==============================] - 241s 33ms/step - loss: 0.0705 - acc: 0.9756 - val_loss: 0.0909 - val_acc: 0.9755
Epoch 5/30
7352/7352 [==============================] - 241s 33ms/step - loss: 0.0722 - acc: 0.9749 - val_loss: 0.0942 - val_acc: 0.9716
Epoch 6/30
7352/7352 [==============================] - 242s 33ms/step - loss: 0.0700 - acc: 0.9754 - val_loss: 0.0961 - val_acc: 0.9700
Epoch 7/30
7352/7352 [==============================] - 240s 33ms/step - loss: 0.0646 - acc: 0.9766 - val_loss: 0.0714 - val_acc: 

In [0]:
# Confusion Matrix
print(confusion_matrix(Y_test, model.predict(X_test)))

Pred                LAYING  SITTING  ...  WALKING_DOWNSTAIRS  WALKING_UPSTAIRS
True                                 ...                                      
LAYING                 537        0  ...                   0                 0
SITTING                  1      448  ...                   0                 6
STANDING                 0      159  ...                   0                10
WALKING                  0        6  ...                  12                 2
WALKING_DOWNSTAIRS       0        0  ...                 416                 2
WALKING_UPSTAIRS         0       15  ...                   7               434

[6 rows x 6 columns]


In [0]:
score = model.evaluate(X_test, Y_test)

2947/2947 [==============================] - 17s 6ms/step


In [0]:
score

[0.13728767729193367, 0.9683293742894075]

## Conclusion

In [3]:
from prettytable import PrettyTable
x = PrettyTable()
x.field_names = ["Model","Description","Dropout","Test loss", "Test Accuracy"]
x.add_row([1,"1 Layer of LSTM(32)",0.5,0.3297, 0.9087])
x.add_row([2,"1 Layer of LSTM(48)",0.5,0.2968, 0.9175])
x.add_row([3,"1 Layer of LSTM(64)",0.25,0.4332, 0.8971])
x.add_row([3,"1 Layer of LSTM(64)",0.5,0.3611, 0.9155])
x.add_row([4,"2 layers of LSTM, BN, Binarycross_entropy",0.7,0.1372, 0.9683])
print(x)

+-------+-------------------------------------------+---------+-----------+---------------+
| Model |                Description                | Dropout | Test loss | Test Accuracy |
+-------+-------------------------------------------+---------+-----------+---------------+
|   1   |            1 Layer of LSTM(32)            |   0.5   |   0.3297  |     0.9087    |
|   2   |            1 Layer of LSTM(48)            |   0.5   |   0.2968  |     0.9175    |
|   3   |            1 Layer of LSTM(64)            |   0.25  |   0.4332  |     0.8971    |
|   3   |            1 Layer of LSTM(64)            |   0.5   |   0.3611  |     0.9155    |
|   4   | 2 layers of LSTM, BN, Binarycross_entropy |   0.7   |   0.1372  |     0.9683    |
+-------+-------------------------------------------+---------+-----------+---------------+
